In [47]:
definitions['Word'].value_counts()

Word
BayesianRidge                     2
AgglomerativeClustering           2
QuadraticDiscriminantAnalysis     1
PowerTransformer                  1
NeighborhoodComponentsAnalysis    1
                                 ..
GaussianProcessRegressor          1
GaussianRandomProjection          1
GenericUnivariateSelect           1
GradientBoostingClassifier        1
Recurrent Neural Network          1
Name: count, Length: 250, dtype: int64

In [46]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category']=='Machine Learning']

from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
d = final_dataset_for_markdown(ml_notes,ml_def)
d.to_csv('/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/d_learning_notes.csv',index=False)

d.head(25)

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
3,Machine Learning,Distinct Types of ML,Supervised Learning,Supervised learning trains a model using label...
4,Machine Learning,Distinct Types of ML,Unsupervised Learning,Unsupervised learning works with unlabeled dat...
5,Machine Learning,Distinct Types of ML,Semisupervised Learning,Semi-supervised learning combines a small amou...
6,Machine Learning,Distinct Types of ML,Reinforcement Learning,Reinforcement learning trains an agent to make...
1,Machine Learning,Optimization,Optimization,Optimization is the process of finding the bes...
7,Machine Learning,Optimization,Goal,"What are you attempting to maximize, Training ..."
8,Machine Learning,Optimization,Approach,Curse of Dimensionality
9,Machine Learning,Optimization,Important to Remember,Must remember what the Model is actually tryin...
10,Machine Learning,Optimization,Important to Remember,Models will cheat and can solve problems which...
